Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [272]:
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse
from functools import reduce
from copy import copy
import math

In [273]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    #inizialize a matrix with all False (by default)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)

    #put True values in the matrix according to the probability in the density
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
            
    #put a True in all the position at least for every position
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [274]:
NUM_POINTS = 100
NUM_SETS = 100
DENSITY = .3

NUM_STEPS = 1_000

In [275]:
SETS = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY)
print("Element at row=42 and column=42:", SETS[42, 42])
print(SETS.toarray()[0])

Element at row=42 and column=42: True
[ True  True False  True False False  True  True False False False False
 False  True False  True False False  True False  True  True  True  True
 False False False False False False  True False  True False  True False
 False False False False False False False False False False False False
 False False  True  True  True False False False  True  True False  True
  True False False False False  True False False  True False  True False
 False False False  True  True False False  True  True  True  True False
 False False False False False  True False  True False False  True False
 False False  True False]


# Common functions

simple

* `fitness`

In [276]:
def fitness1(state):
    cost = sum(state)
    valid = np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS.toarray()[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost

def fitness3(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS.toarray()[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, cost
    

fitness = fitness2

In [277]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_SETS - 1)
    new_state[index] = not new_state[index]
    return new_state

# Hill Climbing (Local search)

In [280]:
def local_search(current_state):
    for step in range(NUM_STEPS):
        new_state = tweak(current_state)
        if fitness(new_state) >= fitness(current_state):
            current_state = new_state
             print(fitness(current_state))

    return current_state


current_state = [False for _ in range(NUM_SETS)]

final_state = local_search(current_state)
print(fitness(final_state))

(31, -1)
(50, -2)
(65, -3)
(82, -4)
(87, -5)
(91, -6)
(94, -7)
(96, -8)
(97, -9)
(98, -10)
(99, -11)
(100, -12)
(100, -11)
(100, -10)


KeyboardInterrupt: 

# Simulated Annealing

In [ ]:
current_state = [False for _ in range(NUM_SETS)]
print(fitness(current_state))

#start from NUM_STEPS+1 and go up to 1 (NUM_STEPS+1 becouse the loop finish at 1 and not 0 so i need to get one back
#  and i finish at 1 and not 0 because it is not possibile to divided by 0)
for time in range(NUM_STEPS+1, 0, -1):
    new_state = tweak(current_state)
    #get actual fitness
    v1, c1 = fitness(current_state) #f (s)
    v2, c2 = fitness(new_state)     #f'(s)
    f = (v1-c1)
    f1 = (v2-c2)
    #calcolate p (probability of accepting a worsening solution)
    p = math.exp(- ((f-f1)/time) ) # p = e^ ((f (s) - f'(s))/time)
    print("==>", p, time)
    if ( fitness(new_state) >= fitness(current_state) ) or (random() < p):
        current_state = new_state
        print(fitness(current_state))

# Tabu Search

In [259]:
NUM_NEIGHBORS = 20
TABU_LIST_MAX_SIZE = 10

In [256]:
def is_valid(state):
    tot, _ = fitness(state)
    if(tot == NUM_POINTS):
        return True
    return False

In [269]:
tabu_list = []
#current_state = [False for _ in range(NUM_SETS)] #hight probability of initial stagnation
current_state = [choice([True, False]) for _ in range(NUM_SETS)]

for iteration in range(NUM_STEPS):
    neighbors = [tweak(current_state) for _ in range(NUM_NEIGHBORS)]
    
    # Select valid candidates not in tabu list
    candidates = [sol for sol in neighbors if is_valid(sol) and sol not in tabu_list]
    
    if not candidates:
        # Any candidates avaiable, it coud be an arrest point
        break
    
    # Valuta candidati e scegli il migliore
    best_candidate = min(candidates, key=fitness3)
    
    # Aggiorna la lista tabù con la mossa corrente
    tabu_list.append(best_candidate)
    
    # Rimuovi le mosse più vecchie dalla lista tabù se è troppo lunga
    if len(tabu_list) > TABU_LIST_MAX_SIZE:
        tabu_list.pop(0)
    
    # Aggiorna la soluzione corrente con il miglior candidato
    current_state = best_candidate

    print(fitness(current_state))


(100, -54)
(100, -53)
(100, -52)
(100, -51)
(100, -50)
(100, -49)
(100, -48)
(100, -47)
(100, -46)
(100, -45)
(100, -44)
(100, -43)
(100, -42)
(100, -41)
(100, -40)
(100, -39)
(100, -38)
(100, -37)
(100, -36)
(100, -35)
(100, -34)
(100, -33)
(100, -32)
(100, -31)
(100, -30)
(100, -29)
(100, -28)
(100, -27)
(100, -26)
(100, -25)
(100, -24)
(100, -23)
(100, -22)
(100, -21)
(100, -20)
(100, -19)
(100, -18)
(100, -17)
(100, -16)
(100, -15)
(100, -14)
(100, -13)
(100, -12)
(100, -11)
(100, -12)
(100, -13)
(100, -12)
(100, -11)
(100, -12)
(100, -13)
(100, -14)
(100, -15)
(100, -14)
(100, -13)
(100, -14)
(100, -13)
(100, -12)
(100, -11)
(100, -10)
(100, -11)
(100, -10)
(100, -11)
(100, -10)
(100, -9)
(100, -10)
(100, -11)
(100, -12)
(100, -11)
(100, -12)
(100, -11)
(100, -12)
(100, -11)
(100, -10)
(100, -9)
(100, -10)
(100, -11)
(100, -12)
(100, -11)
(100, -10)
(100, -11)
(100, -10)
(100, -11)
(100, -12)
(100, -13)
(100, -14)
(100, -13)
(100, -12)
(100, -11)
(100, -10)
(100, -11)
(100, -12)
(

# Iterated Local Search

In [283]:
NUM_ITERATIONS = 10

In [284]:
def new_starting_position(global_, last):
    if global_ is None:
        # Genera una nuova soluzione casuale se la soluzione globale è nulla
        return [choice([True, False]) for _ in range(NUM_SETS)]
    else:
        # Restituisce la soluzione globale corrente se non è nulla
        return global_

In [288]:
global_solution = None

for iteration in range(NUM_ITERATIONS):
    # Fase di miglioramento locale
    current_state = new_starting_position(global_solution, None)
    current_state = local_search(current_state)
    
    # Fase di perturbazione
    new_state = tweak(current_state)
    
    # Seconda fase di miglioramento locale sulla soluzione perturbata
    new_state = local_search(new_state)
    
    # Accettazione della soluzione perturbata in base alla funzione obiettivo
    if fitness(new_state) < fitness(current_state):
        current_state = new_state
    
    # Aggiornamento della soluzione globale se necessario
    if global_solution is None or fitness(current_state) < fitness(global_solution):
        global_solution = current_state
    
    print(fitness(global_solution), iteration)

print(fitness(global_solution))


(100, -49)
(100, -48)
(100, -47)
(100, -46)
(100, -45)
(100, -44)
(100, -43)
(100, -42)
(100, -41)
(100, -40)
(100, -39)
(100, -38)
(100, -37)
(100, -36)
(100, -35)
(100, -34)
(100, -33)
(100, -32)
(100, -31)
(100, -30)
(100, -29)
(100, -28)
(100, -27)
(100, -26)
(100, -25)
(100, -24)
(100, -23)
(100, -22)
(100, -21)
(100, -20)
(100, -19)
(100, -18)
(100, -17)
(100, -16)
(100, -15)
(100, -14)
(100, -13)
(100, -12)
(100, -11)
(100, -10)
(100, -9)
(100, -8)
(100, -8)
(100, -8) 0
(100, -52)
(100, -51)
(100, -50)
(100, -49)
(100, -48)
(100, -47)
(100, -46)
(100, -45)
(100, -44)
(100, -43)
(100, -42)
(100, -41)
(100, -40)
(100, -39)
(100, -38)
(100, -37)
(100, -36)
(100, -35)
(100, -34)
(100, -33)
(100, -32)
(100, -31)
(100, -30)
(100, -29)
(100, -28)
(100, -27)
(100, -26)
(100, -25)
(100, -24)
(100, -23)
(100, -22)
(100, -21)
(100, -20)
(100, -19)
(100, -18)
(100, -17)
(100, -16)
(100, -15)
(100, -14)
(100, -13)
(100, -12)
(100, -11)
(100, -10)
(100, -9)
(100, -9)
(100, -8)
(100, -9) 1
(10